In [ ]:
# %% [markdown]
# # FASE 2: Preprocesamiento y Limpieza de Texto
# ## Juanes - Procesamiento NLP

# %%
# Instalación de dependencias
import sys
import os
sys.path.append('../scripts')

# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from preprocessing import TextPreprocessor

# Configurar estilo de gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Cargar datos crudos
# %%
df = pd.read_csv("../data/comentarios.csv")
df.head()

# Mostrar información básica del dataset
# %%
df.info()
df.describe(include='all')

# Distribución de puntuaciones
# %%
plt.figure(figsize=(6,4))
sns.countplot(x="puntuacion", data=df)
plt.title("Distribución de puntuaciones")
plt.show()


# Ejemplo de texto_comentario_comentarios antes del procesamiento
# %%
df["comentario"].iloc[0]

# Inicializar preprocesador
# %%
preprocessor = TextPreprocessor(language="spanish")

# Procesar una reseña de ejemplo para ver el pipeline
# %%
texto_ejemplo = df["comentario"].iloc[0]
resultado = preprocessor.process(texto_ejemplo)

resultado

# Procesar todo el dataset
# %%
df["tokens_limpios"] = df["comentario"].apply(preprocessor.process)
df["texto_limpio"] = df["tokens_limpios"].apply(lambda x: " ".join(x))

# Calcular métricas adicionales
# %%
df["longitud_original"] = df["comentario"].astype(str).apply(len)
df["longitud_limpia"] = df["texto_limpio"].apply(len)
df["num_palabras"] = df["tokens_limpios"].apply(len)

df["densidad_lexica"] = df["num_palabras"] / df["longitud_original"]

# Guardar dataset procesado
# %%
df.to_csv("../data/comentarios_procesados.csv", index=False)

# Análisis de las métricas calculadas
# %%
df[["longitud_original", "longitud_limpia", "num_palabras", "densidad_lexica"]].describe()

# Estadísticas de longitud
# %%
estadisticas_longitud = pd.DataFrame({
    "Métrica": [
        "Media",
        "Mediana",
        "Desviación estándar",
        "Mínimo",
        "Máximo"
    ],
    "Longitud original": [
        df["longitud_original"].mean(),
        df["longitud_original"].median(),
        df["longitud_original"].std(),
        df["longitud_original"].min(),
        df["longitud_original"].max()
    ],
    "Longitud limpia": [
        df["longitud_limpia"].mean(),
        df["longitud_limpia"].median(),
        df["longitud_limpia"].std(),
        df["longitud_limpia"].min(),
        df["longitud_limpia"].max()
    ],
    "Número de palabras": [
        df["num_palabras"].mean(),
        df["num_palabras"].median(),
        df["num_palabras"].std(),
        df["num_palabras"].min(),
        df["num_palabras"].max()
    ]
})

estadisticas_longitud

In [ ]:
# Visualización de métricas

# 1. Distribución de longitud de texto_comentario_comentarios originales
# %%
sns.histplot(df["longitud_original"], bins=30)
plt.title("Distribución longitud original del texto")
plt.show()

# 2. Distribución de longitud de texto_comentario_comentarios limpios
# %%
sns.histplot(df["longitud_limpia"], bins=30)
plt.title("Distribución longitud texto limpio")
plt.show()

# 3. Distribución de número de palabras
# %%
sns.histplot(df["num_palabras"], bins=30)
plt.title("Distribución número de palabras")
plt.show()

# 4. Densidad léxica por puntuación
# %%
plt.figure(figsize=(8,5))
sns.boxplot(x="puntuacion", y="densidad_lexica", data=df)
plt.title("Densidad léxica por puntuación")
plt.show()

# 5. Relación entre longitud y puntuación
# %%
sns.scatterplot(x="longitud_original", y="puntuacion", data=df)
plt.title("Relación longitud del texto y puntuación")
plt.show()


# 6. Distribución de densidad léxica
# %%
sns.histplot(df["densidad_lexica"], bins=30)
plt.title("Distribución densidad léxica")
plt.show()

In [ ]:
# Análisis de stopwords eliminadas

# Contar frecuencia de palabras antes y después de eliminar stopwords
# %%
texto_original = " ".join(df["comentario"].astype(str))
texto_limpio = " ".join(df["texto_limpio"])

# Combinar todos los texto_comentario_comentarios
# %%
texto_comentarios_original = " ".join(
    df["comentario"].astype(str).tolist()
)

texto_comentarios_limpio = " ".join(
    df["texto_limpio"].tolist()
)

# Contar frecuencias
# %%
from collections import Counter

freq_original = Counter(texto_original.lower().split())
freq_limpio = Counter(texto_limpio.split())

# Mostrar las palabras más comunes antes de eliminar stopwords
# %%
freq_original.most_common(10)
# %%
freq_limpio.most_common(10)


# Preparar datos para la siguiente fase
# %%
df_final = df[[
    "comentario",
    "texto_limpio",
    "tokens_limpios",
    "puntuacion",
    "num_palabras",
    "densidad_lexica"
]]

# Crear dataset optimizado para análisis
# %%
df_analisis = pd.DataFrame({
    "comentario_original": df["comentario"],
    "texto_limpio": df["texto_limpio"],
    "tokens_limpios": df["tokens_limpios"],
    "puntuacion": df["puntuacion"],
    "longitud_original": df["longitud_original"],
    "longitud_limpia": df["longitud_limpia"],
    "num_palabras": df["num_palabras"],
    "densidad_lexica": df["densidad_lexica"]
})

# Guardar dataset para análisis
# %%
df_final.to_csv("../data/comentarios_ready_for_analysis.csv", index=False)

# Resumen del preprocesamiento
# %%
print(f"""
Número total de comentarios: {len(df)}
Longitud media original: {df['longitud_original'].mean():.2f}
Longitud media limpia: {df['longitud_limpia'].mean():.2f}
Número medio de palabras: {df['num_palabras'].mean():.2f}
Densidad léxica media: {df['densidad_lexica'].mean():.3f}
""")